In [ ]:
# :: 10th January 2023

SOLID: L = Liskov substiution principle:
Definition:
- If S is a SUBTYPE of T, then objects of type T may be REPLACED by objects of type S, without distrupting the program.
  - Eg if you have a client code with class T then all its substypes can be used interchangeably by the client without harming the functionality of the client program.

LSP Benefits:
- Client code can use any subtypes of a class interchangeably
- Client is isolated form the class hierachy of the objects it uses.

LSP Violation Problems
- Client depends on concrete implementations
- If a subtype changes, client must be changed
  - IF we don't adhere to the principle we violate the open close principle


Liskov is a supporter for the open closed principle
- If we respect LSP there is a good chance that we respect or adhere to the open close principle.

Spotting LSP violations
- Do subtypes methods have different signatures (do they take different arguments)
- Do subtypes methods have different argument types?

### Violates Liskov LSP principle


In [6]:
from abc import abstractmethod, ABC


class Extractor(ABC): # base class that provides an interface (extract)
    @abstractmethod
    def extract(self, data):
        pass
    
class SpectrogramExtractor(Extractor):
    def extract(self, data):
        print(f'Extracted spectorgram')
    
class MFCCExtractor(Extractor):
    def extract(self, data, num_mfccs):
        print(f'Extracted MFCCs')
        
class MelSpectrogramExtractor(Extractor):
    def extract(self, data):
        print('Extracted mel spectrogram')

class DLPipeline:
    def __init__(self, extractor):
        self.extractor = extractor
    
    def run(self, data):
        print(f'Running DL pipeline')
        features = self._extract(data) # Private method
        # Here implementation of DL steps
        
    def _extract(self, data): # New design does not have access to the strategies in extractor. It just knows the interface.
        self.extractor.extract(data)
            
if __name__ == '__main__':
    dummy_data = [1,2,3]
    mfcc_extractor = MFCCExtractor()
    mfcc_extractor.extract(dummy_data, 35)

Extracted MFCCs


Why does the above violate LSP?
- Do our methods have the same signatures? same arguments? NO.
  - Extract takes 1 argument (data) meanwhile MFCC extractor takes 2 arguments, data and number of mfccs.
  - If we do have client code that uses this class it cannot easily swap one concrete extractor for another as they have different signatures.


### FAKE SOLUTION BELOW FOR POINT ABOVE

In [ ]:
from abc import abstractmethod, ABC


class Extractor(ABC): # base class that provides an interface (extract)
    @abstractmethod
    def extract(self, data, num_coefficients: int):
        pass
    
class MFCCExtractor(Extractor):
    def extract(self, data, num_mfccs: int):
        print(f'Extracted MFCCs')
        
class MelSpectrogramExtractor(Extractor):
    def extract(self, data, num_mels: int):
        print('Extracted mel spectrogram')


class DLPipeline:
    def __init__(self, extractor):
        self.extractor = extractor
    
    def run(self, data):
        print(f'Running DL pipeline')
        features = self._extract(data) # Private method
        # Here implementation of DL steps
        
    def _extract(self, data): # New design does not have access to the strategies in extractor. It just knows the interface.
        self.extractor.extract(data)
            
if __name__ == '__main__':
    dummy_data = [1,2,3]
    mfcc_extractor = MFCCExtractor()
    mfcc_extractor.extract(dummy_data, 35)

We still violate the principle however as the second argument of mel and mfcc are the same number of arguments in the signatures but however they semantically are different things. 
- Thus we subtly violate the principle as the semantic level fails.

### Design that respects Liskov LSP

In [7]:
from abc import abstractmethod, ABC


class Extractor(ABC): # base class that provides an interface (extract)
    @abstractmethod
    def extract(self, data):
        pass
    
class MFCCExtractor(Extractor):
    def __init__(self, num_mfccs):
        self.num_mfccs = num_mfccs
        
    def extract(self, data):
        print(f'Extracted MFCCs')
        
class MelSpectrogramExtractor(Extractor):
    def __init__(self, num_mels):
        self.num_mels = num_mels
        
    def extract(self, data):
        print('Extracted mel spectrogram')


if __name__ == '__main__':
    dummy_data = [1,2,3]
    mfcc_extractor = MFCCExtractor(35)
    mfcc_extractor.extract(dummy_data)

Extracted MFCCs


Now the signatures for all classes takes only data.
All the different methods have the same sigs with the same data.
As a consequence we assign the number of extra params as an attribute at initialiszation time in the constructors.